In [2]:
# brute_force.py

import math
import time
from itertools import permutations


class BruteForce:
    def __init__(self, file_path, cutoff):
        """
        Initialize the BruteForce object with TSP file and cutoff time.
        Args:
            file_path (str): Path to the .tsp file.
            cutoff (float): Time cutoff (in seconds).
        """
        self.file_path = file_path
        self.cutoff = cutoff
        self.coordinates = self._parse_tsp_file()
        self.num_cities = len(self.coordinates)
        self.distances = self._precompute_distances()
        self.best_tour_length = float('inf')

    def _parse_tsp_file(self):
        """
        Parses a .tsp file and extracts the node coordinates.
        Returns:
            list: List of tuples representing coordinates of the locations [(x1, y1), (x2, y2), ...].
        """
        coordinates = []
        with open(self.file_path, 'r') as file:
            lines = file.readlines()

        # Locate the "NODE_COORD_SECTION" and read coordinates
        coordinates_start = lines.index("NODE_COORD_SECTION\n") + 1
        for line in lines[coordinates_start:]:
            if line.strip() == "EOF":
                break
            _, x, y = line.strip().split()
            coordinates.append((float(x), float(y)))

        return coordinates

    def _euclidean_distance(self, p1, p2):
        """Calculate Euclidean distance between two points."""
        return math.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)

    def _precompute_distances(self):
        """Precomputes the distances between all cities."""
        return [[self._euclidean_distance(self.coordinates[i], self.coordinates[j])
                 for j in range(self.num_cities)] for i in range(self.num_cities)]

    def _calculate_tour_length(self, tour):
        """Calculates the total distance of a given tour."""
        return sum(self.distances[tour[i]][tour[(i + 1) % self.num_cities]] for i in range(self.num_cities))

    def find_best_tour_length(self):
        """
        Solves TSP using a brute force approach.
        Returns:
            float: The best tour length found.
        """
        start_time = time.time()
        all_tours = permutations(range(self.num_cities))  # Generate all permutations of cities

        for tour in all_tours:
            # Check time cutoff
            if time.time() - start_time > self.cutoff:
                break

            # Calculate the tour length
            tour_length = self._calculate_tour_length(tour)

            # Update the best tour length
            if tour_length < self.best_tour_length:
                self.best_tour_length = tour_length

        return self.best_tour_length


# Example Usage
# if __name__ == "__main__":
#     file_path = "data/Atlanta.tsp"  # Replace with your .tsp file path
#     cutoff = 60  # Time cutoff in seconds

#     brute_force = BruteForce(file_path, cutoff)
#     best_tour_length = brute_force.find_best_tour_length()
#     print(f"Best Tour Length: {best_tour_length}")


In [4]:
import os
import matplotlib.pyplot as plt

# List of cities and corresponding file paths
CITIES = [
    "Atlanta", "Berlin", "Boston", "Champaign", "Cincinnati", "Denver",
    "NYC", "Philadelphia", "Roanoke", "SanFrancisco", "Toronto",
    "UKansasState", "UMissouri"
]
DATA_DIR = "./"  # Directory where the .tsp files are located
CUTOFF_TIMES = [0, 2, 5, 10, 20, 40, 50, 100, 150, 200, 500]
PLOT_DIR = "./plots"  # Directory to save the plots

# Create the output directory for plots if it doesn't exist
os.makedirs(PLOT_DIR, exist_ok=True)

def run_brute_force_and_plot():
    for city in CITIES:
        tsp_file = os.path.join(DATA_DIR, f"{city}.tsp")
        best_tour_lengths = []

        print(f"Running BruteForce for city: {city}")

        # Run the algorithm for each cutoff time
        for cutoff in CUTOFF_TIMES:
            brute_force_solver = BruteForce(file_path=tsp_file, cutoff=cutoff)
            best_tour_length = brute_force_solver.find_best_tour_length()
            best_tour_lengths.append(best_tour_length)
            print(f"  Cutoff: {cutoff}s, Best Tour Length: {best_tour_length}")

        # Plot the results
        plot_city_results(city, CUTOFF_TIMES, best_tour_lengths)

def plot_city_results(city, cutoff_times, best_tour_lengths):
    plt.figure()
    plt.plot(cutoff_times, best_tour_lengths, marker="o", linestyle="-")
    plt.title(f"{city} - Best Tour Length vs Cutoff Time")
    plt.xlabel("Cutoff Time (s)")
    plt.ylabel("Best Tour Length")
    plt.grid(True)
    plt.savefig(os.path.join(PLOT_DIR, f"{city}_brute_force_plot.png"))
    plt.close()
    print(f"Plot saved for {city}.")

if __name__ == "__main__":
    run_brute_force_and_plot()


Running BruteForce for city: Atlanta
  Cutoff: 0s, Best Tour Length: inf
  Cutoff: 2s, Best Tour Length: 4109866.624376217
  Cutoff: 5s, Best Tour Length: 4109866.624376217
  Cutoff: 10s, Best Tour Length: 3937992.7000216707
  Cutoff: 20s, Best Tour Length: 3816909.110191864
  Cutoff: 40s, Best Tour Length: 3720831.531662631
  Cutoff: 50s, Best Tour Length: 3630532.301182535
  Cutoff: 100s, Best Tour Length: 3630532.301182535
  Cutoff: 150s, Best Tour Length: 3630532.301182535
  Cutoff: 200s, Best Tour Length: 3630532.301182535


KeyboardInterrupt: 

In [5]:
# local_search.py

import math
import random
import time


class LocalSearch:
    def __init__(self, file_path, cutoff, seed):
        """
        Initialize the LocalSearch object with TSP file, cutoff time, and random seed.
        Args:
            file_path (str): Path to the .tsp file.
            cutoff (float): Time cutoff (in seconds).
            seed (int): Random seed for reproducibility.
        """
        self.file_path = file_path
        self.cutoff = cutoff
        self.seed = seed
        self.coordinates = self._parse_tsp_file()
        self.num_cities = len(self.coordinates)
        self.distances = self._precompute_distances()

    def _parse_tsp_file(self):
        """
        Parses a .tsp file and extracts the node coordinates.
        Returns:
            list: List of tuples representing coordinates of the locations [(x1, y1), (x2, y2), ...].
        """
        coordinates = []
        with open(self.file_path, 'r') as file:
            lines = file.readlines()

        # Locate the "NODE_COORD_SECTION" and read coordinates
        coordinates_start = lines.index("NODE_COORD_SECTION\n") + 1
        for line in lines[coordinates_start:]:
            if line.strip() == "EOF":
                break
            _, x, y = line.strip().split()
            coordinates.append((float(x), float(y)))

        return coordinates

    def _euclidean_distance(self, p1, p2):
        """Calculate Euclidean distance between two points."""
        return math.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)

    def _precompute_distances(self):
        """Precomputes the distances between all cities."""
        return [[self._euclidean_distance(self.coordinates[i], self.coordinates[j])
                 for j in range(self.num_cities)] for i in range(self.num_cities)]

    def _calculate_total_distance(self, route):
        """Calculates the total distance of a route."""
        total_distance = sum(self.distances[route[i]][route[i + 1]] for i in range(len(route) - 1))
        total_distance += self.distances[route[-1]][route[0]]  # Return to start
        return total_distance

    def _get_neighbor(self, route):
        """
        Generates a neighboring solution by swapping two cities.
        Args:
            route (list): Current route.
        Returns:
            list: A new route with two cities swapped.
        """
        new_route = route[:]
        i, j = random.sample(range(len(route)), 2)
        new_route[i], new_route[j] = new_route[j], new_route[i]
        return new_route

    def _acceptance_probability(self, old_cost, new_cost, temperature):
        """
        Calculates the probability of accepting a worse solution.
        Args:
            old_cost (float): Current solution cost.
            new_cost (float): New solution cost.
            temperature (float): Current temperature.
        Returns:
            float: Acceptance probability.
        """
        if new_cost < old_cost:
            return 1.0
        return math.exp((old_cost - new_cost) / temperature)

    def find_best_distance(self):
        """
        Solves TSP using Simulated Annealing.
        Returns:
            float: The best distance found.
        """
        random.seed(self.seed)
        cities = list(range(self.num_cities))

        # Initialize variables
        current_route = cities[:]
        random.shuffle(current_route)
        current_distance = self._calculate_total_distance(current_route)

        best_distance = current_distance

        start_time = time.time()
        temperature = 100000.0  # Initial temperature
        cooling_rate = 0.9999  # Cooling rate
        min_temperature = 1e-4  # Stopping temperature
        while time.time() - start_time < self.cutoff and temperature > min_temperature:
            # Generate a neighboring solution
            neighbor_route = self._get_neighbor(current_route)
            neighbor_distance = self._calculate_total_distance(neighbor_route)

            # Decide whether to accept the new solution
            if self._acceptance_probability(current_distance, neighbor_distance, temperature) > random.random():
                current_route = neighbor_route
                current_distance = neighbor_distance

                # Update the best solution
                if current_distance < best_distance:
                    best_distance = current_distance

            # Cool down the temperature
            temperature *= cooling_rate

        return best_distance


# # Example Usage
# if __name__ == "__main__":
#     file_path = "data/Atlanta.tsp"  # Replace with your .tsp file path
#     cutoff = 60  # Time cutoff in seconds
#     seed = 42  # Random seed for reproducibility

#     local_search = LocalSearch(file_path, cutoff, seed)
#     best_distance = local_search.find_best_distance()
#     print(f"Best Distance: {best_distance}")


In [7]:
import os
import matplotlib.pyplot as plt

# List of cities and corresponding file paths
CITIES = [
    "Atlanta", "Berlin", "Boston", "Champaign", "Cincinnati", "Denver",
    "NYC", "Philadelphia", "Roanoke", "SanFrancisco", "Toronto",
    "UKansasState", "UMissouri"
]
DATA_DIR = "./"  # Directory where the .tsp files are located
CUTOFF_TIMES = [
    0, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6,
    0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 1
]
PLOT_DIR = "./plots"  # Directory to save the plots

# Create the output directory for plots if it doesn't exist
os.makedirs(PLOT_DIR, exist_ok=True)

def run_local_search_and_plot():
    for city in CITIES:
        tsp_file = os.path.join(DATA_DIR, f"{city}.tsp")
        best_distances = []

        print(f"Running LocalSearch for city: {city}")

        # Run the algorithm for each cutoff time
        for cutoff in CUTOFF_TIMES:
            local_search_solver = LocalSearch(file_path=tsp_file, cutoff=cutoff, seed=42)
            best_distance = local_search_solver.find_best_distance()
            best_distances.append(best_distance)
            print(f"  Cutoff: {cutoff}s, Best Distance: {best_distance}")

        # Plot the results
        plot_city_results(city, CUTOFF_TIMES, best_distances)

def plot_city_results(city, cutoff_times, best_distances):
    plt.figure()
    plt.plot(cutoff_times, best_distances, marker="o", linestyle="-")
    plt.title(f"{city} - Best Distance vs Cutoff Time")
    plt.xlabel("Cutoff Time (s)")
    plt.ylabel("Best Distance")
    plt.grid(True)
    plt.savefig(os.path.join(PLOT_DIR, f"{city}_local_search_plot.png"))
    plt.close()
    print(f"Plot saved for {city}.")

if __name__ == "__main__":
    run_local_search_and_plot()


Running LocalSearch for city: Atlanta
  Cutoff: 0s, Best Distance: 4504201.150699763
  Cutoff: 0.1s, Best Distance: 2060176.4044786252
  Cutoff: 0.15s, Best Distance: 2023778.354086583
  Cutoff: 0.2s, Best Distance: 2023778.354086583
  Cutoff: 0.25s, Best Distance: 2023778.354086583
  Cutoff: 0.3s, Best Distance: 2023778.354086583
  Cutoff: 0.35s, Best Distance: 2023778.354086583
  Cutoff: 0.4s, Best Distance: 2023778.354086583
  Cutoff: 0.45s, Best Distance: 2023778.354086583
  Cutoff: 0.5s, Best Distance: 2023778.354086583
  Cutoff: 0.55s, Best Distance: 2023778.354086583
  Cutoff: 0.6s, Best Distance: 2023778.354086583
  Cutoff: 0.65s, Best Distance: 2023778.354086583
  Cutoff: 0.7s, Best Distance: 2023778.354086583
  Cutoff: 0.75s, Best Distance: 2023778.354086583
  Cutoff: 0.8s, Best Distance: 2023778.354086583
  Cutoff: 0.85s, Best Distance: 2023778.354086583
  Cutoff: 0.9s, Best Distance: 2023778.354086583
  Cutoff: 1s, Best Distance: 2023778.354086583
Plot saved for Atlanta.
Ru